# Analise de intervencoes do parlamento

## 1.Recolha e formatação de dados

Com base nos dados abertos disponibilizados no site da assembleia da republica, tive a ideia de investigar as varias [intervencoes](http://www.parlamento.pt/Cidadania/Paginas/DAIntervencoes.aspx) correspondentes a ultima legislatura. 

Começamos por importar alguns modulos importantes: 

In [9]:
import json
import pandas as pd
from pprint import pprint
from collections import Counter
from __future__ import division

Após downloadar os dados no formato json, transferimos para um dictionario chamado `interventions`.

In [10]:
with open('data/IntervencoesXIII.json') as data_file:    
    data = json.load(data_file)
    
interventions = data['ArrayOfPt_gov_ar_objectos_intervencoes_DadosPesquisaIntervencoesOut']['pt_gov_ar_objectos_intervencoes_DadosPesquisaIntervencoesOut']
print 'O dictionario tem {} intervencoes.'.format(len(interventions))

O dictionario tem 11072 intervencoes.


Vamos fazer um print da primeira intervenção:

In [11]:
pprint(interventions[0])

{u'actividadeId': u'102944',
 u'actividadesRelacionadas': {u'id': u'102944', u'tipo': u'Disc. Iniciativas'},
 u'dataReuniaoPlenaria': u'2016-07-20',
 u'deputados': {u'GP': u'PS',
                u'idCadastro': u'1600',
                u'nome': u'JAMILA MADEIRA'},
 u'faseSessao': u'POD',
 u'idDebate': u'30572',
 u'iniciativas': {u'pt_gov_ar_objectos_intervencoes_IniciativasOut': {u'fase': u'Vota\xe7\xe3o Delibera\xe7\xe3o',
                                                                      u'id': u'40581',
                                                                      u'tipo': u'Projeto de Resolu\xe7\xe3o'}},
 u'legislatura': u'XIII',
 u'publicacao': {u'pt_gov_ar_objectos_PublicacoesOut': {u'URLDiario': u'http://debates.parlamento.pt/catalogo/r3/dar/01/13/01/089/2016-07-21/115?pgs=115&org=PLC',
                                                        u'idAct': None,
                                                        u'idDeb': None,
                                         

Vamos obrigar as nossa intervencoes a ter alguns campos essenciais.

In [12]:
must_have_fields = ['deputados', 'dataReuniaoPlenaria', 'sumario']

E criamos uma list a partir da qual vamos criar uma Pandas dataframe:

In [13]:
pandas_data = [['id_debate', 'date', 'phase', 'speaker', 'political_party', 'summary_text', 'intervention_type']]

In [20]:
# initialize counters
total = 0
retrieved = 0

# loop over list
for intervention in interventions:
    total += 1
    if all(x in intervention.keys() for x in must_have_fields):
        # define fields
        debate_id = intervention['idDebate']
        date = intervention['dataReuniaoPlenaria']
        phase = intervention['faseSessao']
        speaker = intervention['deputados']['nome']
        political_party = intervention['deputados']['GP']
        summary_text = intervention['sumario']
        intervention_type_text = intervention['tipoIntervencao']
        
        # add to pandas list
        pandas_data += [[debate_id, date, phase, speaker, political_party, summary_text, intervention_type_text]] 
        retrieved += 1

Perdemos? 

In [21]:
print 'Intervencoes parsadas: ', total
print 'Intervencoes que foram aceites: ', retrieved
print 'Perdemos {}%.'.format(round(retrieved/total,2))

Intervencoes parsadas:  11072
Intervencoes que foram aceites:  10593
Perdemos 0.96%.


Criamos dataframe: 

In [22]:
assembleia = pd.DataFrame(pandas_data[1:], columns=pandas_data[0])

## 2.Analise preliminar

Imprimimos a cabeça:

In [24]:
assembleia.head()

,id_debate,date,phase,speaker,political_party,summary_text,intervention_type
0,30572,2016-07-20,POD,JAMILA MADEIRA,PS,Recomenda ao Governo a elaboração de levantame...,Declaração de voto
1,30571,2016-07-20,POD,JOÃO OLIVEIRA,PCP,Regula o acesso à gestação de substituição nos...,Declaração de voto
2,30592,2016-07-20,POD,LUÍS MONTENEGRO,PSD,Cria um regime de reembolso de impostos sobre ...,Declaração de voto
3,30591,2016-07-20,POD,ANA MESQUITA,PCP,"Terceira alteração à Lei n.º 54/2005, de 15 de...",Declaração de voto
4,30590,2016-07-20,POD,ANA MESQUITA,PCP,"Altera a Lei n.º 54/2005, de 15 de novembro, q...",Declaração de voto


In [25]:
assembleia.shape
print 'Temos {} colunas e {} linhas.'.format(assembleia.shape[1], assembleia.shape[0])

Temos 7 colunas e 63558 linhas.


Tipos de dados? 

In [26]:
assembleia.dtypes

id_debate            object
date                 object
phase                object
speaker              object
political_party      object
summary_text         object
intervention_type    object
dtype: object

Quais os diferentes partidos e tipos de intervenções? 

In [30]:
for partido in list(assembleia.political_party.unique()):
    print partido

PS
PCP
PSD
CDS-PP
PEV
BE
PAN


In [32]:
for tipo_de_intervencao in list(assembleia.intervention_type.unique()):
    print tipo_de_intervencao

Declaração de voto
Declaração politica
Defesa da honra
Explicações
Interpelação à mesa
Intervenção
Leitura
Pedido de esclarecimento
Pergunta
Réplica
Defesa da consideração
Recurso da decisão da mesa


In [35]:
assembleia.political_party.value_counts(normalize=True)

PS        0.249221
PSD       0.237421
PCP       0.151798
CDS-PP    0.146606
BE        0.137355
PEV       0.052204
PAN       0.025394
Name: political_party, dtype: float64

In [36]:
assembleia.intervention_type.value_counts(normalize=True)

Intervenção                   0.490796
Declaração de voto            0.207968
Pedido de esclarecimento      0.165770
Leitura                       0.046918
Pergunta                      0.039366
Interpelação à mesa           0.028037
Declaração politica           0.014349
Explicações                   0.003304
Defesa da honra               0.001794
Réplica                       0.001510
Defesa da consideração        0.000094
Recurso da decisão da mesa    0.000094
Name: intervention_type, dtype: float64

In [37]:
assembleia.speaker.value_counts(normalize=True)

HELOÍSA APOLÓNIA             0.027754
ANDRÉ SILVA                  0.025394
JOSÉ LUÍS FERREIRA           0.024450
PEDRO FILIPE SOARES          0.016992
ANTÓNIO FILIPE               0.016143
JOÃO OLIVEIRA                0.015199
NUNO MAGALHÃES               0.014444
CARLA CRUZ                   0.013783
ANTÓNIO CARLOS MONTEIRO      0.013499
PAULA SANTOS                 0.013405
BRUNO DIAS                   0.013216
DUARTE PACHECO               0.012744
EDUARDO FERRO RODRIGUES      0.012650
MOISÉS FERREIRA              0.012555
RITA RATO                    0.012555
JOÃO RAMOS                   0.012461
IDÁLIA SALVADOR SERRÃO       0.012272
CECÍLIA MEIRELES             0.010667
MARIANA MORTÁGUA             0.010573
JOANA MORTÁGUA               0.010290
ANA MESQUITA                 0.010290
HELDER AMARAL                0.010101
ANA RITA BESSA               0.009912
CARLOS CÉSAR                 0.009912
PEDRO MOTA SOARES            0.009535
VÂNIA DIAS DA SILVA          0.009440
JOSÉ MOURA S

In [22]:
assembleia[assembleia.political_party == 'PCP'].intervention_type.value_counts(normalize=True)

Intervenção                   0.544776
Pedido de esclarecimento      0.195896
Declaração de voto            0.174751
Pergunta                      0.036070
Interpelação à mesa           0.021766
Declaração politica           0.017413
Leitura                       0.003731
Explicações                   0.001866
Réplica                       0.001866
Defesa da consideração        0.000622
Defesa da honra               0.000622
Recurso da decisão da mesa    0.000622
Name: intervention_type, dtype: float64

In [23]:
assembleia[assembleia.political_party == 'PSD'].intervention_type.value_counts(normalize=True)

Intervenção                 0.414712
Declaração de voto          0.291849
Pedido de esclarecimento    0.153877
Interpelação à mesa         0.048907
Leitura                     0.046123
Pergunta                    0.027038
Declaração politica         0.010736
Réplica                     0.002783
Defesa da honra             0.002386
Explicações                 0.001590
Name: intervention_type, dtype: float64

In [24]:
assembleia[assembleia.political_party == 'PS'].intervention_type.value_counts(normalize=True)

Intervenção                 0.394318
Declaração de voto          0.305682
Pedido de esclarecimento    0.129167
Leitura                     0.096970
Pergunta                    0.032576
Interpelação à mesa         0.021591
Declaração politica         0.011364
Explicações                 0.006818
Defesa da honra             0.001515
Name: intervention_type, dtype: float64

In [25]:
assembleia[assembleia.political_party == 'BE'].intervention_type.value_counts(normalize=True)

Intervenção                 0.596564
Pedido de esclarecimento    0.225430
Declaração de voto          0.065979
Pergunta                    0.040550
Leitura                     0.030241
Interpelação à mesa         0.017182
Declaração politica         0.016495
Explicações                 0.004811
Réplica                     0.002062
Defesa da honra             0.000687
Name: intervention_type, dtype: float64

In [26]:
assembleia.political_party.unique()

array([u'PS', u'PCP', u'PSD', u'CDS-PP', u'PEV', u'BE', u'PAN'],
      dtype=object)

In [ ]:
assembleia.political_party.unique()